## yaml2json.py

In [ ]:
import json
import os
import sys
import yaml


thisdir = os.getcwd()
final=str(thisdir)+'\yamls'
arr = os.listdir(final)

for i in arr:
    a = os.path.join(final, i)
    print(a)
    if os.path.exists(a):
        source_file = open(a, "r")
        source_content = yaml.safe_load(source_file)
        source_file.close()
    else:
        print("ERROR: " + str(i)+ " not found")
        exit(1)

    # Processing the conversion
    output = json.dumps(source_content,ensure_ascii=False, indent=2)
    b='jsons/json_normal/'+str(i) + '.json'
    target_file = open(b, "w")
    target_file.write(output)
    target_file.close()

## convert.py

In [ ]:
import json
import sys
import os

thisdir = os.getcwd()
final=str(thisdir)+'\jsons\json_normal'
arr = os.listdir(final)

for i in arr:
    a = os.path.join(final, i)
    print(a)
    with open(a, 'r') as f:
        intents = json.load(f)

    dict1={}
    dict2={}
    list1 = []
    pattern_list = []
    responses_list = []
    i=0
    for key,value in intents.items():
        if key == 'conversations':
            for i in range((len(intents['conversations']))):
                pattern_list = []
                responses_list = []
                tag = str(str(intents['categories'][0])) +" "+str(i)
                patterns = (intents['conversations'][i][:1])
                responses = (intents['conversations'][i][1:])
                print(patterns,tag,responses,i)
                pattern_list.append(patterns)
                responses_list.append(responses)
                dict2 = {"tag":tag,"patterns":patterns,"responses":responses}
                list1.append(dict2)
                print(list1)
        if key == 'categories':

            key = str(intents['categories'][0])
            a='jsons/json_convert/'+str(key) + '.json'
            # dict1.__setitem__(str(key),list1)
            dict1.__setitem__("intents",list1)
    print(intents.keys())



    with open(a, 'w') as outfile:
        json.dump(dict1, outfile,ensure_ascii=False, indent=1)

## merge.py

In [ ]:
import json
import sys
import os
from time import sleep
from tqdm import tqdm
import os

thisdir = os.getcwd()
final=str(thisdir)+'\jsons\json_convert'
arr = os.listdir(final)

# python merge.py intents_new.json
tags=[]
patterns=[]
responses=[]

with open('jsons/master_intents.json', 'r') as f:
    intents = json.load(f)


# a=sys.argv[1]
for i in arr:
    a = os.path.join(final, i)
    print(a)
    with open(a, 'r') as f:
        intents1 = json.load(f)

    for i in intents['intents']:
        tags.append(i['tag'])
        patterns.append(i['patterns'])
        responses.append(i['responses'])

    for i in intents1['intents']:
        tags_x=i['tag']
        if tags_x in tags:
            x=tags.index(tags_x)
            y=intents['intents'][x]
            for j in i['patterns']:
                if j not in patterns[x]:
                    y['patterns'].append(j)
            for k in i['responses']:
                if k not in responses[x]:
                    y['responses'].append(k)
        else:
            dict1={'tag': i['tag'],
                'patterns': i['patterns'],
                'responses': i['responses'],
            }
            intents['intents'].append(dict1)
            

# for i in tqdm(range(len(intents['intents']))):
#     sleep(1)

with open('jsons/sample_intents.json', 'w') as outfile:
    json.dump(intents, outfile,ensure_ascii=False, indent=4)

# os.remove(a)
# print("File Removed")

## model.py

In [3]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

## train.py 

In [4]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

with open('jsons/master_intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

449 patterns
427 tags: ['AI 0', 'AI 1', 'AI 10', 'AI 11', 'AI 12', 'AI 13', 'AI 14', 'AI 15', 'AI 16', 'AI 17', 'AI 18', 'AI 19', 'AI 2', 'AI 20', 'AI 21', 'AI 22', 'AI 23', 'AI 24', 'AI 25', 'AI 26', 'AI 27', 'AI 28', 'AI 29', 'AI 3', 'AI 30', 'AI 31', 'AI 32', 'AI 33', 'AI 34', 'AI 35', 'AI 36', 'AI 37', 'AI 38', 'AI 39', 'AI 4', 'AI 40', 'AI 41', 'AI 42', 'AI 43', 'AI 44', 'AI 45', 'AI 46', 'AI 47', 'AI 48', 'AI 49', 'AI 5', 'AI 50', 'AI 51', 'AI 52', 'AI 53', 'AI 54', 'AI 55', 'AI 56', 'AI 57', 'AI 58', 'AI 59', 'AI 6', 'AI 7', 'AI 8', 'AI 9', 'computers 0', 'computers 1', 'computers 2', 'computers 3', 'computers 4', 'computers 5', 'computers 6', 'computers 7', 'computers 8', 'computers 9', 'conversations 0', 'conversations 1', 'conversations 10', 'conversations 11', 'conversations 12', 'conversations 13', 'conversations 14', 'conversations 15', 'conversations 16', 'conversations 17', 'conversations 18', 'conversations 19', 'conversations 2', 'conversations 20', 'conversations 21',

Epoch [100/1000], Loss: 0.6668
Epoch [200/1000], Loss: 0.0216
Epoch [300/1000], Loss: 0.0014
Epoch [400/1000], Loss: 0.0006
Epoch [500/1000], Loss: 0.0001
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 1.3136
Epoch [1000/1000], Loss: 0.0000
final loss: 0.0000
training complete. file saved to data.pth


## model_save.py 

In [6]:
import random
import json
import pandas as pd
import torch

from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()
print(model.eval)



pd.to_pickle(model,r'chat_model.pickle')

FileNotFoundError: [Errno 2] No such file or directory: 'intents.json'

## run.py

In [ ]:
import os
print ("Starting script1")
os.system("python yaml2json.py")
print ("script1 ended")
print ("Starting script2")
os.system("python convert.py")
print ("script2 ended")
print ("Starting script3")
os.system("python merge.py")
print ("script3 ended")
print ("Starting script4")
os.system("python train.py")
print ("script4 ended")

## chat_API.py

In [ ]:
import random
import json

import torch

from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('jsons/sample_intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Thor"
print("Let's chat! (type 'quit' to exit)")
# while True:
#     # sentence = "do you use credit cards?"
#     sentence = input("You: ")
#     if sentence == "quit":
#         break
def brain(sentence):
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                # print(f"{bot_name}: {random.choice(intent['responses'])}")
                response = random.choice(intent['responses'])
                return response

    else:
        # print(f"{bot_name}: I do not understand...")
        response = "I do not understand..."
        return response

## app1.py

In [ ]:
from flask import Flask,render_template, jsonify, request,redirect, url_for
from flask_cors import CORS
import chat_API
import pywhatkit
import pickle
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
CORS(app)
# run_with_ngrok(app)               #this is needed to use ngrok for port forwarding
database={'subho':'hero','soham':'chomu','yash':'pagla','tublu':'7415'}

@app.route('/predict',methods=['GET','POST'])
def index():
    user_input = request.args.get('user_input')
    user_input = user_input.lower()
    print(user_input)
    if "search youtube" in user_input:
        user_input = user_input.replace('search youtube', '')  
        pywhatkit.playonyt(user_input)
        return jsonify({'user_input':str("searched youtube"+str(user_input))})
    elif "search" in user_input:
        user_input = user_input.replace('search', '')  
        pywhatkit.search(user_input) 
        return jsonify({'user_input':str("searched"+str(user_input))})
    else:
        return jsonify({'user_input':str(chat_API.brain(user_input))})

if __name__ == '__main__':
    app.run()

## chat.py

In [1]:
import random
import json

import torch

from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('jsons/master_intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Thor"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hi
Thor: Hi there, how can I help?
Thor: i am mighty THOR
You: tell me a joke
Thor: what do you get when you cross a cow and a lemon?
You: quit


## app.py

In [2]:
from flask import Flask,render_template, jsonify, request,redirect, url_for
from flask_cors import CORS
import chat_API
import pywhatkit
import pickle
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)               #this is needed to use ngrok for port forwarding
database={'subho':'hero','soham':'chomu','yash':'pagla','tublu':'7415'}

# @app.route('/predict',methods=['POST'])
# def index():
#     user_input = request.args.get('user_input')
#     return jsonify({'user_input':str(chat_API.brain(user_input))})

@app.route("/")
def home():
    return render_template("login.html")

@app.route('/form_login',methods=['POST','GET'])
def login():
    name1=request.form['username']
    pwd=request.form['password']
    if name1 not in database:
	    return render_template('login.html',info='Invalid User')
    else:
        if database[name1]!=pwd:
            return render_template('login.html',info='Invalid Password')
        else:
	        return render_template('index.html',name=name1)



@app.route("/get")
def get_bot_response():
    userText = request.args.get('user_input')
    userText = userText.lower()
    if "search youtube" in userText:
        userText = userText.replace('search youtube', '')  
        pywhatkit.playonyt(userText)
        return "searched youtube"+str(userText)
    elif "search" in userText:
        userText = userText.replace('search', '')  
        pywhatkit.search(userText) 
        return "searched"+str(userText)
    elif "whatsapp" in userText:
        return "I am trying" + "I cannot open whatsapp"
    elif "what is your name" in userText:
        return "My name is Chatterbot"
    return str(chat_API.brain(userText))
    
@app.route('/logout')
def logout():
    return redirect(url_for('home'))


if __name__ == '__main__':
    app.run()

Let's chat! (type 'quit' to exit)
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2021 14:51:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 14:51:33] "GET /favicon.ico HTTP/1.1" 404 -


 * Running on http://ab9af3844279.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
